In [1]:
from argparse import Namespace
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
np.random.seed(1337)

args = Namespace(
    TIME_STEPS = 20,  # same as the height of the image/ rows
    INPUT_SIZE = 1,  # same as the width of the image/ columns
    BATCH_SIZE = 50,
    BATCH_START = 0,
    OUTPUT_SIZE = 1,
    CELL_SIZE = 20,
    EPOCHES = 501,
    LR = 1e-3,
)

In [3]:
def get_batch(args):
    xs = np.arange(args.BATCH_START,
                   args.BATCH_START+args.TIME_STEPS*args.BATCH_SIZE
         ).reshape((args.BATCH_SIZE, args.TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    
    args.BATCH_START += args.TIME_STEPS
    
    plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    plt.show()
    
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

In [4]:
def build_model(args):
    model = Sequential()
    
    model.add(LSTM(
        batch_input_shape=(args.BATCH_SIZE, args.TIME_STEPS, args.INPUT_SIZE),
        output_dim=args.CELL_SIZE,
        return_sequences=True,
        stateful=True,  # batches are related
    ))
    
    model.add(TimeDistributed(Dense(args.OUTPUT_SIZE)))
    
    return model

In [5]:
def train(args):
    print('Training', '-' * 10)
    fig,ax = plt.subplots(1,1)
    ax.set_ylim(-1.2, 1.2)
    for step in range(args.EPOCHES):
        X_batch, y_batch, xs = get_batch(args)
        cost = model.train_on_batch(X_batch, y_batch)
        pred = model.predict(X_batch, args.BATCH_SIZE)

        
        ax.plot(xs[0, :], y_batch[0].flatten(), 'r',
                  xs[0, :], pred.flatten()[:args.TIME_STEPS], 'b--')

        fig.canvas.draw()

        if step % 10 == 0:
            print('train cost:', cost)

In [6]:
# predict cos according to sin

adam = Adam(args.LR)
model = build_model(args)
model.compile(optimizer=adam, loss='mse')

Instructions for updating:
Colocations handled automatically by placer.


/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(50, 20, 1..., return_sequences=True, stateful=True, units=20)`
  


In [ ]:
%matplotlib inline
train(args)